In [8]:
import pandas as pd
import pickle
import pprint

In [6]:
%%time

individual_provider_payment_df_clean = pd.read_pickle("../data/individual_provider_payment.pkl")

Wall time: 16.9 s


In [14]:
individual_provider_payment_df_clean.describe()

,National Provider Identifier,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount
count,9.416125e+06,9.416125e+06,9.416125e+06,9.416125e+06,9.416125e+06,9.416125e+06,9.416125e+06,9.416125e+06
mean,1.499785e+09,1.983183e+02,7.084164e+01,1.151297e+02,9.735926e+01,3.316574e+02,7.392625e+01,7.453863e+01
std,2.877289e+08,2.351712e+03,1.442648e+02,2.783331e+02,2.186003e+02,8.947265e+02,1.705221e+02,1.706421e+02
min,1.003000e+09,2.400000e+00,1.100000e+01,1.100000e+01,6.035380e-05,6.035380e-05,0.000000e+00,0.000000e+00
25%,1.255302e+09,2.000000e+01,1.700000e+01,2.000000e+01,2.501000e+01,6.000000e+01,1.967050e+01,2.020000e+01
50%,1.497969e+09,4.300000e+01,3.200000e+01,4.000000e+01,6.639000e+01,1.490000e+02,4.745194e+01,4.811000e+01
75%,1.740681e+09,1.160000e+02,7.300000e+01,1.040000e+02,1.131400e+02,2.997289e+02,8.456263e+01,8.464778e+01
max,1.993000e+09,3.301134e+06,9.532700e+04,1.758700e+05,3.986417e+04,9.999900e+04,3.125351e+04,3.145565e+04


In [11]:
individual_provider_payment_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9416125 entries, 0 to 9416124
Data columns (total 23 columns):
 #   Column                                                    Dtype  
---  ------                                                    -----  
 0   National Provider Identifier                              int64  
 1   Last Name/Organization Name of the Provider               object 
 2   First Name of the Provider                                object 
 3   Credentials of the Provider                               object 
 4   Entity Type of the Provider                               object 
 5   Street Address 1 of the Provider                          object 
 6   City of the Provider                                      object 
 7   Zip Code of the Provider                                  object 
 8   State Code of the Provider                                object 
 9   Country Code of the Provider                              object 
 10  Provider Type                 

In [10]:
individual_provider_payment_df_clean['Provider Type'].value_counts()

Diagnostic Radiology                   1241400
Internal Medicine                      1118171
Family Practice                         969268
Nurse Practitioner                      560219
Cardiology                              445088
                                        ...   
Ambulance Service Provider                  42
Unknown Supplier/Provider Specialty         17
All Other Suppliers                         11
Slide Preparation Facility                   2
Medical Toxicology                           2
Name: Provider Type, Length: 88, dtype: int64

In [15]:
individual_provider_payment_df_clean['HCPCS Description'].value_counts()

Established patient office or other outpatient visit, typically 15 minutes                                            457835
Established patient office or other outpatient, visit typically 25 minutes                                            428262
New patient office or other outpatient visit, typically 45 minutes                                                    179569
New patient office or other outpatient visit, typically 30 minutes                                                    178057
Subsequent hospital inpatient care, typically 25 minutes per day                                                      175335
                                                                                                                       ...  
Removal of soft tissue at roof of mouth                                                                                    1
Gene analysis (serpin peptidase inhibitor, clade A, alpha-1 antiproteinase, antitrypsin, member 1) common variants         1


In [13]:
pd.pivot_table(individual_provider_payment_df_clean,index=["Provider Type", "HCPCS Description"])

Average Medicare Allowed Amount  \
Provider Type      HCPCS Description                                                                     
Addiction Medicine Administration of influenza virus vaccine                                   24.1125   
                   Administration of pneumococcal vaccine                                      22.7150   
                   Alcohol and/or substance (other than tobacco) a...                          69.6200   
                   Analysis of urine                                                            2.9700   
                   Annual depression screening, 15 minutes                                     17.5150   
...                                                                                                ...   
Vascular Surgery   X-ray of spine of neck, 2 or 3 views                                        10.7400   
                   X-ray of upper arm, minimum of 2 views                                       8.0100   
                   X-ray of upper spine, 4 or 5 views                                          15.0400   
                   X-ray of upper spine, 6 or more views                                       17.4200   
                   X-ray of wrist, minimum of 3 views                                           8.3600   

                                                                       Average Medicare Payment Amount  \
Provider Type      HCPCS Description                                                                     
Addiction Medicine Administration of influenza virus vaccine                                 23.632500   
                   Administration of pneumococcal vaccine                                    22.260000   
                   Alcohol and/or substance (other than tobacco) a...                        53.877956   
                   Analysis of urine                                                          2.910000   
                   Annual depression screening, 15 minutes                                   17.000000   
...                                                                                                ...   
Vascular Surgery   X-ray of spine of neck, 2 or 3 views                                       7.976842   
                   X-ray of upper arm, minimum of 2 views                                     5.273571   
                   X-ray of upper spine, 4 or 5 views                                        10.575417   
                   X-ray of upper spine, 6 or more views                                     13.660000   
                   X-ray of wrist, minimum of 3 views                                         6.055385   

                                                                       Average Medicare Standardized Amount  \
Provider Type      HCPCS Description                                                                          
Addiction Medicine Administration of influenza virus vaccine                                      25.320000   
                   Administration of pneumococcal vaccine                                         25.320000   
                   Alcohol and/or substance (other than tobacco) a...                             55.292240   
                   Analysis of urine                                                               2.910000   
                   Annual depression screening, 15 minutes                                        17.940000   
...                                                                                                     ...   
Vascular Surgery   X-ray of spine of neck, 2 or 3 views                                            8.564737   
                   X-ray of upper arm, minimum of 2 views                                          5.760714   
                   X-ray of upper spine, 4 or 5 views                                             11.177917   
                   X-ray of upper spine, 6 or more views                                    